# **Proceso main: Selenium Finviz & API FMD**

In [1]:
def get_estimations(ticker):
    """
    
        Fetch financial estimations from Financial Modeling Prep API
        
        Parameters:
            ticker: str - The stock ticker symbol to search for.
        Returns:
            Dictionary with the fetched estimations (key-value pairs including metrics like price target, earnings estimates, etc.)
    
    """
    
    # Ensure necessary imports
    import requests
    
    # Financial Modeling Prep (FMP) API key for estimations
    API_KEY = 'm6B6VyNRoaMYJOIxJPWLzD6K9oVopgoe'
    
    # Prepare the URLs
    URL_FINANCIAL_ESTIMATES = f'https://financialmodelingprep.com/stable/analyst-estimates'
    params_financial_estimates = {
        'apikey': API_KEY,
        'symbol': ticker.upper(),
        'period': 'annual'
    }
    
    URL_PRICE_TARGET_CONSENSUS = f'https://financialmodelingprep.com/stable/price-target-consensus'
    params_price_target_consensus = {
        'apikey': API_KEY,
        'symbol': ticker.upper()
    }
    
    URL_STOCK_GRADES = f'https://financialmodelingprep.com/stable/grades-consensus'
    params_stock_grades = params_price_target_consensus
    
    # Function to fetch data
    def fetch_data(url, params):
        try:
            # Make the GET request to the API endpoint
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            # Process the response data
            if isinstance(data, list):
                return data[0] if data else {}
            
            if isinstance(data, dict):
                return data
            
            # Handle unexpected data format
            print(f"Unexpected data format from {url}: {type(data)}")
            return {}
        
        # Handle request exceptions
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching data from {url}: {e}")
            return {}
    
    # Fetch data from the three endpoints
    financial_estimates = fetch_data(URL_FINANCIAL_ESTIMATES, params_financial_estimates)
    price_target_consensus = fetch_data(URL_PRICE_TARGET_CONSENSUS, params_price_target_consensus)
    stock_grades = fetch_data(URL_STOCK_GRADES, params_stock_grades)
    
    # Return combined data
    return {**financial_estimates, **price_target_consensus, **stock_grades}
    
def get_information(ticker):
    """
    
        Selenium Web Scraping from finviz.com
        
        Parameters:
            ticker: str - The stock ticker symbol to search for.
        Returns:
            Dictionary with the extracted information (key-value pairs including metrics like P/E ratio, market cap, etc.)
    
    """
    
    # Ensure necessary imports
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import time
    
    # Initialize the WebDriver and information dictionary
    driver = webdriver.Chrome()
    info = {}
    
    # Full screen the window
    driver.maximize_window()
    
    try:
        # Access the website
        driver.get("https://finviz.com/")
        
        # Handle cookie consent pop-up
        time.sleep(1)  # Wait for the pop-up to appear
        cookie_reject_button = driver.find_element(By.CLASS_NAME, "Button__StyledButton-buoy__sc-a1qza5-0")
        cookie_reject_button.click()
        
        # Locate the search input field and enter the ticker symbol. Then wait 0.5 seconds and press ENTER
        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.TAG_NAME, "input")))
        search_input.click()
        search_input.send_keys(ticker.upper())
        time.sleep(0.5)
        search_input.send_keys(Keys.ENTER)
        
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "snapshot-table2")))
        
        # Extract the required information
        table = driver.find_element(By.CLASS_NAME, "snapshot-table2")
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Iterate through rows and extract key-value pairs
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            for i in range(0, len(cells), 2):
                key = cells[i].text
                value = cells[i + 1].text
                info[key] = value
    
    # Handle exceptions
    except Exception as e:
        print(f"An error occurred: {e}")
    
    # Ensure the driver is closed properly
    finally:
        time.sleep(1)
        driver.quit()
    
    # Return the extracted information
    return info

if __name__ == "__main__":
    import time
    
    t0 = time.time()
    ticker_symbol = input("Enter the ticker symbol: ")
    info = get_information(ticker_symbol)
    estimations = get_estimations(ticker_symbol)
    data = {**info, **estimations}
    t1 = time.time()

    print(f"Time taken: {t1 - t0} seconds")
    
    for key, value in data.items():
        print(f"{key}: {value}")

Time taken: 18.829941987991333 seconds
Index: DJIA, NDX, S&P 500
P/E: 44.83
EPS (ttm): 4.04
Insider Own: 4.06%
Shs Outstand: 24.30B
Perf Week: 3.41%
Market Cap: 4398.06B
Forward P/E: 23.59
EPS next Y: 64.10%
Insider Trans: -1.01%
Shs Float: 23.31B
Perf Month: -2.96%
Enterprise Value: 4348.27B
PEG: 0.49
EPS next Q: 1.51
Inst Own: 68.18%
Short Float: 1.00%
Perf Quarter: -1.43%
Income: 99.20B
P/S: 23.50
EPS this Y: 56.40%
Inst Trans: -0.06%
Short Ratio: 1.22
Perf Half Y: 25.54%
Sales: 187.14B
P/B: 37.00
ROA: 77.15%
Short Interest: 232.90M
Perf YTD: 34.78%
Book/sh: 4.89
P/C: 72.57
EPS next 5Y: 48.51%
ROE: 107.36%
52W High: 212.19 -14.70%
Perf Year: 38.81%
Cash/sh: 2.49
P/FCF: 56.88
EPS past 3/5Y: 96.93% 91.83%
ROIC: 77.27%
52W Low: 86.62 108.95%
Perf 3Y: 924.05%
Dividend Est.: 0.04 (0.02%)
EV/EBITDA: 38.58
Sales past 3/5Y: 69.25% 64.24%
Gross Margin: 70.05%
Volatility: 2.56% 3.28%
Perf 5Y: 1259.93%
Dividend TTM: 0.04 (0.02%)
EV/Sales: 23.24
EPS Y/Y TTM: 58.53%
Oper. Margin: 58.84%
ATR (14)

In [ ]:
def generate_financial_summary(raw_data):
    """
    
        Filter financial data and generate a formatted table with key metrics and interpretations.
        
        Parameters:
            raw_data: dict - Dictionary containing raw financial metrics from web scraping and API calls.
        Returns:
            String containing a formatted table with filtered metrics, their values, and economic interpretations.
    
    """
    
    # Ensure necessary imports
    from tabulate import tabulate
    
    # Definition of metrics to filter and their explanations
    # Format: 'Original_Key': ('Readable_Name', 'Description/Importance')
    metrics_map = {
        'Price': ('Current Price', 'Current market value of the stock.'),
        'Market Cap': ('Market Capitalization', 'Total size of the company in the market.'),
        'Perf Year': ('Annual Performance', 'Percentage change in stock price over the last year.'),
        'P/E': ('P/E Ratio', 'Price/Earnings. Indicates how expensive the stock is.'),
        'Forward P/E': ('Forward P/E', 'Expected Price/Earnings ratio (lower values indicate potential improvement).'),
        'Target Price': ('Target Price', 'Price that analysts expect within 12 months.'),
        'consensus': ('Consensus', 'Average analyst opinion (Buy/Hold/Sell).'),
        'targetHigh': ('Analysts Ceiling', 'The most optimistic price target recorded.'),
        'EPS next Y': ('EPS Growth', 'Expected earnings growth for the next year.'),
        'ROE': ('ROE (%)', 'Return on Equity. Measures efficiency of capital use.'),
        'Debt/Eq': ('Debt/Capital', 'Leverage level. Low values indicate financial strength.'),
        'Profit Margin': ('Profit Margin', 'Percentage of revenue converted to profit.'),
        'RSI (14)': ('RSI Index', 'Indicates if the stock is overbought (>70) or oversold (<30).'),
        'revenueAvg': ('Revenue 2030 (Est)', 'Long-term revenue projection according to API.'),
        'numAnalystsEps': ('Number of Analysts', 'Quantity of experts covering this company.')
    }

    table_data = []
    
    # Filtering process (requirement for extra points)
    for key, (label, description) in metrics_map.items():
        value = raw_data.get(key, "N/A")
        table_data.append([label, value, description])

    # Table generation with tabulate
    headers = ["Metric", "Value", "Economic Interpretation"]
    
    return tabulate(table_data, headers=headers, tablefmt='rounded_grid')


t = generate_financial_summary(data)

print(t)

╭───────────────────┬──────────────┬──────────────────────────────────────────────────────────────────────╮
│ Métrica           │ Valor        │ Interpretación Económica                                             │
├───────────────────┼──────────────┼──────────────────────────────────────────────────────────────────────┤
│ Precio Actual     │ 180.99       │ Valor de mercado de la acción actualmente.                           │
├───────────────────┼──────────────┼──────────────────────────────────────────────────────────────────────┤
│ Capitalización    │ 4398.06B     │ Tamaño total de la empresa en el mercado.                            │
├───────────────────┼──────────────┼──────────────────────────────────────────────────────────────────────┤
│ Ratio P/E         │ 44.83        │ Precio/Beneficio. Indica cómo de cara está la acción.                │
├───────────────────┼──────────────┼──────────────────────────────────────────────────────────────────────┤
│ Rendimiento Anual │ 38.81%

In [23]:
print(type(data))

print(len(data))

<class 'dict'>
115


## **Intento de webscrapping mediante `BeautifulSoup & requests` pero recibía error 401 (No autorizado)**

In [ ]:
def get_estimations(ticker):
    # Ensure necessary imports
    from bs4 import BeautifulSoup
    import requests
    
    url = f"https://marketwatch.com/investing/stock/{ticker.lower()}/analystestimates"
    
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
                "Accept-Language": "en-US,en;q=0.9",
                "Referrer": "https://google.com"}
    
    try:
        response = requests.get(url, headers=headers)
    
        if response.status_code != 200:
            print(f"Failed to retrieve data for {ticker.upper()}. Status code: {response.status_code}")
            return {}
    
        soup = BeautifulSoup(response.content, "html.parser")
        estimations = {}
    
        try:
            tables = soup.find_all("table", class_ = "table value-pairs no-heading font--lato")
            
            for i in range(2):
                for row in tables[i].find_all("tr"):
                    cells = row.find_all("td")
                    if len(cells) >= 2:
                        key = cells[0].get_text(strip=True)
                        value = cells[1].get_text(strip=True)
                        estimations[key] = value
            
            if estimations:
                print(f"Estimations for {ticker.upper()} extracted successfully")
            else:
                print(f"No estimations found for {ticker.upper()}")

        except Exception as e:
            print(f"An error occurred while parsing the estimations: {e}")

    except Exception as e:
        print(f"An error occurred while extracting estimations: {e}")
    
    return estimations


## **Intento de webscrapping mediante `Selenium` pero detecta bot**

In [ ]:
def get_estimations(driver, ticker):
    # Ensure necessary imports
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    # Go directly to the MarketWatch analyst estimates page for the ticker
    url = f"https://www.marketwatch.com/investing/stock/{ticker.lower()}/analystestimates"
    estimations = {}

    try:
        driver.get(url)
        
        print("Acceded to MarketWatch analyst estimates page")
        time.sleep(1) # Wait for the page to load
        
        # Cookie pop-up handling 
        try:
            # Wait until appears the iframe containing the cookie button
            iframe = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//iframe[contains(@id, 'sp_message_iframe_1396985')]")))
            
            # Switch to the iframe
            driver.switch_to.frame(iframe)
            
            # Wait for the cookie button to be clickable and click it
            cookie_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text() = 'YES, I AGREE']")))
            cookie_btn.click()
            
            print("Cookie pop-up closed.")
        
        except Exception as e:
            print("No cookie pop-up found or already closed. Exception:", e)

        # Ensure we are at the main content
        driver.switch_to.default_content()

        # Wait for the estimation tables to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "table value-pairs no-heading font--lato")))

        # Find all the estimation tables
        tables = driver.find_elements(By.CLASS_NAME, "table value-pairs no-heading font--lato")
        
        # For the first two tables, extract key-value pairs
        for i in range(2):
            rows = tables[i].find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    estimations[key] = value
        
        # If no estimations were found, log a warning
        if not estimations:
            print(f"Warning: No data scraped for {ticker}")
        else:
            print(f"Successfully scraped analyst data for {ticker}")

    except Exception as e:
        print(f"Error scraping MarketWatch for {ticker}: {e}")

    return estimations